In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

#----------------------------------------------------------
#參數
KEY = 'gold_label'
NUM_WORDS = 1200
MAX_LENGTH = 500
HIDDEN_SIZE = 256
OUTPUT_DIM = 64
BATCH_SIZE = 100
EPOCHS = 20
VALIDATION_SPLIT = 0.15
#----------------------------------------------------------
FILE_PATH = 'snli_1.0_train.csv'
TEST_FILE_PATH = 'snli_1.0_test.csv'
raw_data = pd.read_csv(FILE_PATH, encoding = "ISO-8859-1")  
raw_test_data  = pd.read_csv(TEST_FILE_PATH, encoding = "ISO-8859-1")

print("Raw data loaded successfully...\n")
#資料處理
data_0 = raw_data[raw_data[KEY] == 'neutral']
data_1 = raw_data[raw_data[KEY] == 'entailment']
data_2 = raw_data[raw_data[KEY] == 'contradiction']
raw_data[KEY] = raw_data[KEY].map({'neutral':0,'entailment':1,'contradiction':2}).astype(int)

raw_data = pd.concat([data_0,data_1], axis = 0, ignore_index=True)
raw_data = pd.concat([raw_data,data_2], axis = 0, ignore_index=True)

test_data_0 = raw_test_data[raw_test_data[KEY] == 'neutral'] 
test_data_1 = raw_test_data[raw_test_data[KEY] == 'entailment'] 
test_data_2 = raw_test_data[raw_test_data[KEY] == 'contradiction'] 
raw_test_data[KEY] = raw_test_data[KEY].map({'neutral':0,'entailment':1,'contradiction':2}).astype(int)

raw_data = pd.concat([data_0,data_1], axis = 0, ignore_index=True)
raw_test_data = pd.concat([raw_test_data,data_2], axis = 0, ignore_index=True)

train_data = raw_data.sample(frac = 0.75, random_state = 1)
test_data = raw_test_data.sample(frac = 0.75, random_state = 1)

train_x = train_data['sentence1_parse']
train_x['sentence1_parse']  = train_x['sentence2_parse']
test_x = test_data['sentence1_parse']
test_x['sentence1_parse']  = test_x['sentence2_parse']

train_y = to_categorical(train_data[KEY])
test_y = to_categorical(test_data[KEY])

token = Tokenizer(num_words = NUM_WORDS)
token.fit_on_texts(train_x)

train_x = sequence.pad_sequences(token.texts_to_sequences(train_x),maxlen = MAX_LENGTH)
test_x = sequence.pad_sequences(token.texts_to_sequences(test_x),maxlen = MAX_LENGTH)
#模型建立
model = Sequential()

model.add(Embedding(output_dim = OUTPUT_DIM, input_dim = 4000, input_length = MAX_LENGTH))
model.add(Dropout(0.2))
#LSTM
model.add(LSTM(OUTPUT_DIM))
#HIDDEN_LAYERS
model.add(Dense(units = HIDDEN_SIZE, activation = 'relu'))
model.add(Dropout(0.2))
#OUTPUT_LAYERS
model.add(Dense(units = 4, activation = 'sigmoid'))
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', mertics = ['accuracy'])

train_history = model.fit(x = train_x,y = train_y,validation_data= test_data,epochs= EPOCHS, batch_size = BATCH_SIZE,verbose = 2)

Raw data loaded successfully...



ValueError: Cannot convert non-finite values (NA or inf) to integer